### EXAMEN - Convocatoria 1 - Desarrollo
Utilizar el conjunto de datos "dataset_exam.npy" para resolver el ejercicio. Tener en cuenta que la última columna corresponde a la clase.

#### 1) Partición de datos externa (1.5 puntos)
Realizar una partición externa de tipo hold-out seleccionando un 20% de los datos para test (fijar una semilla en 42).
Comprobar si los datos, tanto de train como de test, están balanceados.

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

datos = np.load('dataset_exam.npy')
X = datos[:,:-1]
y = datos [:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

val, ocur = np.unique(X_train, return_counts=True)
val_test, ocur_test = np.unique(X_test, return_counts=True)

val, ocur = np.unique(y_train, return_counts=True)
val_test, ocur_test = np.unique(y_test, return_counts=True)


#### 2) Estandarización de los datos de train y test (1.5 puntos)
Utilizar el método StandardScaler().

In [25]:
from sklearn import preprocessing

standardizer = preprocessing.StandardScaler()
standardizer.fit(X_train)
X_train_std = standardizer.transform(X_train)
X_test_std = standardizer.transform(X_test)

print(np.shape(X_train_std))
print(np.shape(X_test_std))


(393, 8)
(99, 8)


#### 3) Selección de atributos en train y test (1.5 puntos)
Aplicar el método de información mutua para clasificación en el percentile 90.

In [26]:
from sklearn.feature_selection import SelectPercentile, mutual_info_regression

X_train_info = SelectPercentile(mutual_info_classif, percentile=90).fit_transform(X_train_std, y_train)
X_test_info = SelectPercentile(mutual_info_classif, percentile=90).fit_transform(X_test_std, y_test)


#### 4) Comparación de modelos de clasifición mediante validación cruzada (2.5 puntos)
Aplicar una validación cruzada interna de K=5 bolsas para optimizar y comparar la capacidad predictiva de los siguientes modelos: Regresión Logística y Support Vector Machine. La comparación debe realizarse únicamente en términos de exactitud proporcionando resultados de media +- desviación estándar.

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn import preprocessing, svm

algoritmos = {'LOGR': LogisticRegression(penalty='l2', solver='saga', max_iter=1000, random_state=42),
             'SVM': svm.SVC(C=10, random_state=42, probability=True)}

results={}
for nombre, alg in algoritmos.items():
    results[nombre] = cross_val_score(alg, X_train_info, y_train, cv=KFold(n_splits=5, shuffle=True, random_state=42))
    print(nombre + ':  Accuracy:  %0.4f +/- %0.4f'% (results[nombre].mean(), results[nombre].std()))

LOGR:  Accuracy:  0.6766 +/- 0.0538
SVM:  Accuracy:  0.8039 +/- 0.0376


#### 5) Evaluación de los modelos sobre el conjunto de test (3 puntos)
- Entrenar los modelos anteriores utilizando todos los datos de entrenamiento.
- Evaluar su rendimiento sobre el conjunto de test.
- Mostrar en una única figura la comparación de las curvas ROC obtenidas para cada modelo.
- Crear una tabla donde se muestren los resultados de todos los modelos.
 * Las filas serán: Precisión, Sensibilidad, F-score, Exactitud y AUC
 * Las columnas serán: LR y SVM
- Mostrar la matriz de confusión proporcionada por el mejor modelo. 